Ques 1. a.Divide the dataset into input features (all columns except price) and output variable  
(price)

In [3]:
import pandas as pd
df = pd.read_csv("USA_Housing.csv")

In [4]:
X = df.drop('Price', axis=1)  # input feature without price
y = df['Price']  # output only price
print("Input Features (X):")
display(X.head())

print("\nOutput Variable (y):")
display(y.head())

Input Features (X):


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.45857,5.682861,7.009188,4.09,23086.80050
1,79248.64245,6.002900,6.730821,3.09,40173.07217
2,61287.06718,5.865890,8.512727,5.13,36882.15940
3,63345.24005,7.188236,5.586729,3.26,34310.24283
4,59982.19723,5.040555,7.839388,4.23,26354.10947



Output Variable (y):


,Price
0,1.059034e+06
1,1.505891e+06
2,1.058988e+06
3,1.260617e+06
4,6.309435e+05


b. Scale the values of input features.  

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#to fit and transform the input features only
X_scaled = scaler.fit_transform(X)
print("Scaled Input Features are:")
display(pd.DataFrame(X_scaled, columns=X.columns).head())

Scaled Input Features are:


,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,1.028660,-0.296927,0.021274,0.088062,-1.317599
1,1.000808,0.025902,-0.255506,-0.722301,0.403999
2,-0.684629,-0.112303,1.516243,0.930840,0.072410
3,-0.491499,1.221572,-1.393077,-0.584540,-0.186734
4,-0.807073,-0.944834,0.846742,0.201513,-0.988387


c. Divide input and output features into five folds.

In [6]:
from sklearn.model_selection import KFold

# KFold with 5 splits-
kf = KFold(n_splits=5, shuffle=True, random_state=42)
X_folds = [] #list for storing
y_folds = []

for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_folds.append({'train': X_train, 'test': X_test})
    y_folds.append({'train': y_train, 'test': y_test})

# Display the shape of the first fold's training and testing sets
print("Shape of the first fold's training set (X_train):", X_folds[0]['train'].shape)
print("Shape of the first fold's testing set (X_test):", X_folds[0]['test'].shape)
print("Shape of the first fold's training set (y_train):", y_folds[0]['train'].shape)
print("Shape of the first fold's testing set (y_test):", y_folds[0]['test'].shape)

Shape of the first fold's training set (X_train): (4000, 5)
Shape of the first fold's testing set (X_test): (1000, 5)
Shape of the first fold's training set (y_train): (4000,)
Shape of the first fold's testing set (y_test): (1000,)


d.Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score for each iteration using least square error fit.

In [7]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score

beta_matrices = [] #har fold ka result store krne ke liye list
predicted_values = []
r2_scores = []

for i in range(5):
    X_train = X_folds[i]['train']
    X_test = X_folds[i]['test']
    y_train = y_folds[i]['train']
    y_test = y_folds[i]['test']
    X_train_intercept = np.insert(X_train, 0, 1, axis=1)
    #formula of beta = (X^T * X)^(-1) * X^T * y
    beta = np.linalg.pinv(X_train_intercept.T @ X_train_intercept) @ X_train_intercept.T @ y_train
    beta_matrices.append(beta)
    X_test_intercept = np.insert(X_test, 0, 1, axis=1)
    y_pred = X_test_intercept @ beta
    predicted_values.append(y_pred)

    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    print(f"Iteration {i+1}:")
    print("Beta Matrix (coefficients):", beta)
    print("R2 Score:", r2)
    print("-" * 30)

Iteration 1:
Beta Matrix (coefficients): [1232002.6748241   230745.94073479  163243.27314515  120309.77397759
    3011.45976111  151552.63069359]
R2 Score: 0.9179971706985147
------------------------------
Iteration 2:
Beta Matrix (coefficients): [1232037.85755946  229081.97914235  165882.1605634   121536.57475055
    2092.4478622   150874.99274586]
R2 Score: 0.9145677884802818
------------------------------
Iteration 3:
Beta Matrix (coefficients): [1231951.92563846  230224.50511001  162766.17455493  121022.77324578
    1247.16258975  150234.77720419]
R2 Score: 0.9116116385364479
------------------------------
Iteration 4:
Beta Matrix (coefficients): [1232751.46486511  229500.10043209  165212.07110924  122839.9376815
    3063.71699324  150917.88484984]
R2 Score: 0.9193091764960815
------------------------------
Iteration 5:
Beta Matrix (coefficients): [1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]
R2 Score: 0.9243869413350316
--------------

e.Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data and test the performance for remaining 30% data.

In [8]:
from sklearn.model_selection import train_test_split
best_r2_index = np.argmax(r2_scores)
best_beta = beta_matrices[best_r2_index]

print(f"The best R2 score is {r2_scores[best_r2_index]:.4f} from Iteration {best_r2_index + 1}.")
print("The corresponding Beta Matrix is:", best_beta)

#spliting into 70% training and 30% testing
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

X_test_30_intercept = np.insert(X_test_30, 0, 1, axis=1)
y_pred_30 = X_test_30_intercept @ best_beta

print("\nShape of 70% training set (X_train_70):", X_train_70.shape)
print("Shape of 30% testing set (X_test_30):", X_test_30.shape)
print("Shape of predicted values for 30% test set (y_pred_30):", y_pred_30.shape)

The best R2 score is 0.9244 from Iteration 5.
The corresponding Beta Matrix is: [1.23161736e+06 2.30225051e+05 1.63956839e+05 1.21115120e+05
 7.83467170e+02 1.50662447e+05]

Shape of 70% training set (X_train_70): (3500, 5)
Shape of 30% testing set (X_test_30): (1500, 5)
Shape of predicted values for 30% test set (y_pred_30): (1500,)


Ques 2. Concept of Validation set for Multiple Linear Regression (Gradient Descent Optimization)
Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%).

In [9]:
X = np.c_[np.ones((X.shape[0], 1)), X]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.56, random_state=42)
val_size = 14 / (14 + 30)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=val_size, random_state=42)
print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

Shapes: (2800, 6) (700, 6) (1500, 6)


Define the gradient descent function.

In [11]:
def gradient_descent(X,y,lr,n=1000):
  m,n = X.shape
  beta = np.zeros((n,1))
  for i in range(n):
    gradients = 2 / m * X.T @ (X @ beta - y)
    beta = beta - lr * gradients
  return beta

Compute regression coefficients for different learning rates and evaluate performance on validation and test sets.

In [16]:
learning_rates = [0.001, 0.01, 0.1, 1]
results = []
for lr in learning_rates:
  # y_reshaped = y.values.reshape(-1, 1)
  beta = gradient_descent(X_train, y_train.values.reshape(-1,1), lr)
  y_preds = X_test @ beta
  y_val_preds = X_val @ beta

  r2_val = r2_score(y_val, y_val_preds)
  r2_test = r2_score(y_test, y_preds)

  results.append((lr, beta, r2_val, r2_test))
  print(f"LR={lr}: Validation R2={r2_val:.4f}, Test R2={r2_test:.4f}")

LR=0.001: Validation R2=-142820889120600189474310697200040530578778941973773459965759832145682552865991352647680.0000, Test R2=-140457981155972525052747313122155334014366075269930722956793055306840704820674897641472.0000
LR=0.01: Validation R2=-142821015736657279244528725971737067368982341536305069408514468754858221279286299555896309445558272.0000, Test R2=-140458105677223369186874099527356723010236388574882959881751832973486813664472903672994008678793216.0000
LR=0.1: Validation R2=-142821028398269257915909314138914182881259480770768688582696821773238839688797955803932014784183136119511056384.0000, Test R2=-140458118129354606059640075819222340199492274714826879336271156300552868474877142496601822971659131422207115264.0000
LR=1: Validation R2=-142821029664429319345092687280553554031633611260585985641737518099099816816926757578102337521400221223428208072346998669312.0000, Test R2=-140458119374566631412823522104108198432066395752290180437410399494724790563810719034763576783712642463295421

Q3: Pre-processing and Multiple Linear Regression
Download and load the car price prediction dataset.

In [17]:
car_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data")

Replace '?' with NaN values.

In [18]:
car_data_clean = car_data.replace('?',np.nan)

Assign column names to the dataset.

In [19]:
car_data_clean.columns = ["symboling", "normalized_losses",
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

Display the head of the cleaned dataset.

In [20]:
car_data_clean.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250


Fill missing numerical values with the mean.

In [22]:
for col in car_data_clean.select_dtypes(include=[np.number]).columns:
    if col != "price":
        car_data_clean[col] = car_data_clean[col].fillna(car_data_clean[col].mean())

Drop rows with missing values in the 'price' column.

In [23]:
car_data_clean.dropna(subset=['price'],inplace=True)

Map text values to numerical values for 'num_doors' and 'num_cylinders'.

In [24]:
num_map = {"two": 2, "three": 3, "four": 4, "five": 5,
           "six": 6, "eight": 8, "twelve": 12}
car_data_clean["num_doors"] = car_data_clean["num_doors"].map(num_map).astype(float)
car_data_clean["num_cylinders"] = car_data_clean["num_cylinders"].map(num_map).astype(float)

Apply one-hot encoding to 'body_style' and 'drive_wheels' columns.

In [25]:
car_data_clean = pd.get_dummies(car_data_clean,columns=["body_style","drive_wheels"],drop_first=True)

Encode other categorical features using LabelEncoder.

In [27]:
from sklearn.preprocessing import LabelEncoder

for col in ["make", "aspiration", "engine_location", "fuel_type"]:
    le = LabelEncoder()
    car_data_clean[col] = le.fit_transform(car_data_clean[col].astype(str))

Apply transformation to 'fuel_system' and 'engine_type' columns.

In [28]:
for col in ["fuel_system", "engine_type"]:
    car_data_clean[col] = car_data_clean[col].apply(lambda x: 1 if isinstance(x, str) and ("pfi" in x or "ohc" in x) else 0)

Drop any remaining rows with missing values.

In [29]:
car_data_clean.dropna(inplace=True)

Separate features (X) and target variable (y), and scale the features.

In [30]:
X = car_data_clean.drop(columns=["price"])
y = car_data_clean["price"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Split the data into training and testing sets and train a Linear Regression model without PCA.

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.7, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_original = r2_score(y_test, y_pred)
print("R2 without PCA:", r2_original)

R2 without PCA: 0.7489597315563958


Apply PCA to reduce dimensionality and train a Linear Regression model with PCA.

In [33]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # keep 95% variance
X_reduced = pca.fit_transform(X_scaled)

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reduced, y, train_size=0.7, random_state=42)

model_pca = LinearRegression()
model_pca.fit(X_train_r, y_train_r)

y_pred_pca = model_pca.predict(X_test_r)
r2_pca = r2_score(y_test_r, y_pred_pca)
print("R2 with PCA:", r2_pca)

R2 with PCA: 0.7821519144392621
